# Загрузка библиотек

In [1]:
import pandas as pd

import numpy as np

from catboost import CatBoostClassifier

import sys
import os
sys.path.append(os.path.abspath('./src'))
from preprocessing import load_train_data, run_preproc, load_train_data_with_enc
from scorer import make_pred, get_feature_import

In [2]:
test = pd.read_csv('./input/test.csv')

In [3]:
test.head(1)

,transaction_time,merch,cat_id,amount,name_1,name_2,gender,street,one_city,us_state,post_code,lat,lon,population_city,jobs,merchant_lat,merchant_lon
0,2019-09-14 02:46,"fraud_Stokes, Christiansen and Sipes",grocery_net,25.79,Michael,Rodriguez,M,172 Paula Inlet Apt. 650,Cross Plains,TX,76443,32.1482,-99.1872,1897,Chief Operating Officer,31.772057,-99.103183


In [4]:
train, enc = load_train_data_with_enc()

In [5]:
from category_encoders.cat_boost import CatBoostEncoder
import category_encoders as ce
cbe_encoder = ce.cat_boost.CatBoostEncoder()
cat_col = list(train.select_dtypes([object]).columns)
train_cbe = cbe_encoder.fit_transform(train[cat_col], train['target']) 


In [6]:
for col in cat_col:
    train[col]=train_cbe[col]

In [7]:
dt = run_preproc(train, test, enc)

In [8]:
dt.shape, train.shape

((262144, 42), (786431, 43))

# Обучение модели

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    train.drop(columns='target'), train['target'], test_size=0.2, random_state=42,
    stratify=train['target']
)

In [11]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, random_state=14, custom_metric='F1', l2_leaf_reg=0.1, verbose=250)

model.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	learn: 0.4463679	test: 0.4464409	best: 0.4464409 (0)	total: 140ms	remaining: 2m 19s
250:	learn: 0.0074452	test: 0.0098141	best: 0.0098141 (250)	total: 11.1s	remaining: 33s
500:	learn: 0.0055361	test: 0.0094115	best: 0.0094112 (499)	total: 21.5s	remaining: 21.4s
750:	learn: 0.0041572	test: 0.0093446	best: 0.0093165 (596)	total: 31.8s	remaining: 10.6s
999:	learn: 0.0031863	test: 0.0094155	best: 0.0093165 (596)	total: 42.2s	remaining: 0us

bestTest = 0.009316465037
bestIteration = 596

Shrink model to first 597 iterations.


In [12]:
model.get_best_score()

{'learn': {'F1': 0.9257992565055762, 'Logloss': 0.003186320576473112},
 'validation': {'F1': 0.7363184079601991, 'Logloss': 0.009316465037138904}}

In [13]:
model.save_model('./models/cbClassifier.cbm')